# Detection with YOLOv5

A majority of this tutorial is adapted from the [YOLOv5 Custom Training Notebook](https://colab.research.google.com/github/roboflow-ai/yolov5-custom-training-tutorial/blob/main/yolov5-custom-training.ipynb) and this [Paperspace Blog](https://blog.paperspace.com/train-yolov5-custom-data/).

## Install Requirements

In [1]:
!git clone https://github.com/ultralytics/yolov5 ../../external/yolov5
!pip install -qr ../../external/yolov5/requirements.txt

fatal: destination path '../../external/yolov5' already exists and is not an empty directory.


In [2]:
%load_ext autoreload
%autoreload 2

## Assemble Our Dataset

### Convert the Annotations into the YOLO v5 Format

In this part, we convert annotations into the format expected by YOLO v5.

Annotations for the SoccerTrack dataset are stored in the SoccerTrack format which is primarily for the purpose of tracking.

<img src="https://soccertrack.readthedocs.io/en/latest/_images/soccertrack_dataframe.png" width="640"/>


Below is an example of the SoccerTrack format.

* One row per Multiple objects
* Each object has class bb_left, bb_top, bb_width, bb_height, conf and so on..
* Box coordinates are not normalized
* Each object can be identified by the TeamID and PlayerID

In [3]:
import soccertrack
from soccertrack.logging import show_df # This just makes the df viewable in the notebook.

dataset_path = soccertrack.datasets.get_path('top-view')
path_to_csv = sorted(dataset_path.glob('annotations/*.csv'))[0]

bbdf = soccertrack.load_df(path_to_csv)

show_df(bbdf.head())

YOLOv5 expects annotations for each image in form of a .txt file where each line of the text file describes a bounding box. Consider the following image.

<img src="https://blog.paperspace.com/content/images/2021/03/image-25.png" width="640"/>



The annotation file for the image above looks like the following:

<img src="https://blog.paperspace.com/content/images/2021/03/image-26.png" width="640"/>

There are 3 objects in total (2 persons and one tie). Each line represents one of these objects. The specification for each line is as follows.

* One row per object
* Each row is class x_center y_center width height format.
* Box coordinates must be normalized by the dimensions of the image (i.e. have values between 0 and 1)
* Class numbers are zero-indexed (start from 0).

We can convert the annotation into the YOLO v5 format using the `to_yolov5_format` function provided by SoccerTrack.

In [25]:
from pathlib import Path
import numpy as np
from soccertrack import Camera

# Specify where to save the video
save_dir = Path('../../data/yolov5/' )

# Create a camera object to read the width and height of the video
path_to_mp4 = sorted(dataset_path.glob('videos/*.mp4'))[0]
cam = Camera(path_to_mp4)

yolov5_formatted_data = bbdf.to_yolov5_format(save_dir=save_dir, w=cam.w, h=cam.h)

print('Shape of yolov5_formatted_data:', np.array(yolov5_formatted_data).shape)
print('Sample of yolov5_formatted_data:', yolov5_formatted_data[0][:5])
print(f'Contents of {save_dir}:', *[p.name for p in save_dir.iterdir()][:5], '...')

Shape of yolov5_formatted_data: (900, 23, 5)
Sample of yolov5_formatted_data: [[0.         0.386458   0.50925939 0.010187   0.018111  ]
 [0.         0.505208   0.50000039 0.010187   0.018111  ]
 [0.         0.213948   0.4959345  0.010188   0.018111  ]
 [0.         0.496932   0.24689839 0.010187   0.018111  ]
 [0.         0.413417   0.65064839 0.010187   0.018111  ]]
Contents of ../../data/yolov5: 000534.txt 000252.txt 000246.txt 000520.txt 000508.txt ...


We also want to split the video into individual frames. Iterate through each frame with `iter_frames` and save the image file to the appropriate directory.

In [26]:
import cv2
from tqdm import tqdm

for frame_num, frame in enumerate(tqdm(cam.iter_frames())):
    file_path = f'{save_dir}/{frame_num:06d}.png'
    cv2.imwrite(file_path, frame)

900it [03:35,  4.17it/s]


## Partition the Dataset

In [28]:
from sklearn.model_selection import train_test_split

# Read images and annotations
images = sorted(save_dir.glob('*.png'))
annotations = sorted(save_dir.glob('*.txt'))

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

# Move each file to their respective folder
def move_files_to_folder(files, folder):
    folder.mkdir(exist_ok=True, parents=True)
    for file in files:
        file.rename(folder / file.name)

move_files_to_folder(train_images, save_dir / 'images/train')
move_files_to_folder(val_images, save_dir / 'images/val')
move_files_to_folder(test_images, save_dir / 'images/test')
move_files_to_folder(train_annotations, save_dir / 'labels/train')
move_files_to_folder(val_annotations, save_dir / 'labels/val')
move_files_to_folder(test_annotations, save_dir / 'labels/test')

## Train Our Custom YOLOv5 model

### Data Config File

Details for the dataset you want to train your model on are defined by the data config YAML file. 
The following parameters have to be defined in a data config file:


1. `train`, `test`, and `val`: Locations of train, test, and validation images.
2. `nc`: Number of classes in the dataset.
3. `names`: Names of the classes in the dataset. The index of the classes in this list would be used as an identifier for the class names in the code.

Below we create a new file called `soccertrack_data.yaml` and place it in the `yolov5/data` folder. 

In [29]:
import yaml

dict_file = {
    'train': str((save_dir / 'images/train').absolute()),
    'val': str((save_dir / 'images/val').absolute()),
    'test': str((save_dir / 'images/test').absolute()),
    'nc': 2,
    'names': ['player', 'ball']
}

yaml_path = (save_dir / 'soccertrack_data.yaml').absolute()
with open(yaml_path, 'w') as file:
    documents = yaml.dump(dict_file, file)

### Choosing the Network Architecture

YOLOv5 provides a variety of network architectures. From smallest to largest, they are: `yolov5n`, `yolov5s`, `yolov5m`, `yolov5l`, and `yolov5x`. The larger the network, the more accurate the model, but the slower it will be.

YOLOv5 will automatically download the pre-trained weights for the network architecture you choose.

### Start training the model

Finally, start training the model by running the following command.

In [ ]:
!python ../../external/yolov5/train.py --img 640 --cfg yolov5s.yaml --batch 4 --epochs 100 --data {yaml_path} 

Since you are running this code in a notebook, it would be better to run the training in a separate terminal. We have provided the scripts necessary to do this in the `scripts` folder.

Run the following command in a terminal to start training the model.

```bash
source scripts/yolov5/train.sh
```

## Inference

In [ ]:
import torch

model = torch.hub.load("yolov5/torch/hub/ultralytics_yolov5_master", 'custom', path="best.pt", source='local')